In [5]:
import json

with open('politicianNodes.json', 'r') as f:
    data = json.load(f)

for node in data['nodes']:
    node['id'] = node.pop('key')
    #del node['label']
    node.update(node.pop('attributes'))

with open('politicianNodes_modified.json', 'w') as f:
    json.dump(data, f)